In [1]:
import torch
import random
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# load custom modules required for jetCLR training
from modules.transformer import Transformer
from modules.neural_net import create_and_run_nn



# set gpu device
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")
print( "device: " + str( device ), flush=True)

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.cuda.empty_cache()


from numba import cuda 
os.environ["CUDA_VISIBLE_DEVICES"]="3"
device = cuda.get_current_device()
device.reset()


device: cuda


# Load in the processed data

In [2]:
path_to_save_dir = "/global/home/users/rrmastandrea/training_data/"

sig_samp_id = "CWoLa_n_sig_25000_n_bkg_0_n_nonzero_50_n_pad_0_n_jet_2/"
bkg_samp_id = "CWoLa_n_sig_0_n_bkg_25000_n_nonzero_50_n_pad_0_n_jet_2/"

#TEST_dir = "STANDARD_TEST_SET_n_sig_10k_n_bkg_10k_n_nonzero_50_n_pad_0_n_jet_2/"


n_constits_max = 50
n_jets = 2

path_to_sig_data = path_to_save_dir+sig_samp_id
print(path_to_sig_data)
path_to_bkg_data = path_to_save_dir+bkg_samp_id
print(path_to_bkg_data)

sig_data = np.load(path_to_sig_data+"data_train.npy")
sig_labels = np.load(path_to_bkg_data+"labels_train.npy")
bkg_data = np.load(path_to_bkg_data+"data_train.npy")
bkg_labels = np.load(path_to_bkg_data+"labels_train.npy")

# print data dimensions
print( "Sig data shape: " + str( sig_data.shape ), flush=True)
print( "Sig labels shape: " + str( sig_labels.shape ), flush=True)
print( "Bkg data shape: " + str( bkg_data.shape ), flush=True)
print( "Sig data shape: " + str( bkg_labels.shape ), flush=True)

/global/home/users/rrmastandrea/training_data/CWoLa_n_sig_25000_n_bkg_0_n_nonzero_50_n_pad_0_n_jet_2/
/global/home/users/rrmastandrea/training_data/CWoLa_n_sig_0_n_bkg_25000_n_nonzero_50_n_pad_0_n_jet_2/
Sig data shape: (25000, 3, 102)
Sig labels shape: (25000,)
Bkg data shape: (25000, 3, 102)
Sig data shape: (25000,)


# Run the data through the transformer net

In [3]:
# Load in the transformer net

model_dim = 128

# set gpu device
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")
print( "device: " + str( device ), flush=True)

exp_id = "SB_ratios_22_18_01/16kS_16kB_"+str(model_dim)+"d/"
base_dir = "/global/home/users/rrmastandrea/MJetCLR/"  # change this to your working directory
expt_dir = base_dir + "projects/rep_learning/experiments/" + exp_id + "/"


constit_num = 50

input_dim = 3
output_dim = model_dim
dim_feedforward = model_dim
n_heads = 4
n_layers = 2
n_head_layers = 2
opt = "adam"
learning_rate_trans = 0.0001



loaded_net_BC = Transformer( input_dim, model_dim, output_dim, n_heads, dim_feedforward, 
                  n_layers, learning_rate_trans, n_head_layers, dropout=0.1, opt=opt, BC = True )

loaded_net_BC.load_state_dict(torch.load(expt_dir+"final_model_BC_"+str(constit_num)+".pt"))
loaded_net_BC.eval()

loaded_net_BC.to( device )





device: cuda


Transformer(
  (embedding): Linear(in_features=3, out_features=128, bias=True)
  (decoder): Linear(in_features=128, out_features=1, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128

In [ ]:
# Running the final transformer on the binary classification data

print("Loading data into net...")
lct_train_reps = F.normalize( loaded_net.forward_batchwise( torch.Tensor( data_train ).transpose(1,2), data_train.shape[0], use_mask=mask, use_continuous_mask=cmask ).detach().cpu(), dim=-1  ).numpy()
lct_val_reps = F.normalize( loaded_net.forward_batchwise( torch.Tensor( data_val ).transpose(1,2), data_test_f.shape[0], use_mask=mask, use_continuous_mask=cmask ).detach().cpu(), dim=-1  ).numpy()
lct_test_reps = F.normalize( loaded_net.forward_batchwise( torch.Tensor( data_test_f ).transpose(1,2), data_test_f.shape[0], use_mask=mask, use_continuous_mask=cmask ).detach().cpu(), dim=-1  ).numpy()


In [ ]:
# Create mixed latent-space samples

In [ ]:
# Run CWoLA

## Generate the signal and background samples

In [ ]:
sig_mean = 5
sig_std = 5
num_sig = 10000

sig_sample = np.random.normal(sig_mean, sig_std, num_sig)

bkg_mean = 10
bkg_std = 5
num_bkg = 10000

bkg_sample = np.random.normal(bkg_mean, bkg_std, num_bkg)

data_sample = np.concatenate((sig_sample, bkg_sample))

plt.figure()
plt.hist(data_sample, bins = 50)
plt.show()

In [ ]:
def generate_mixed_sample(sig_set, bkg_set, N, f):
    
    """
    INPUTS
    N: number of total events
    f: signal fraction
    
    OUTPUTS
    mixed_sample : unshuffled array of [signal, background] samples
    """
    
    num_sig = int(f*N)
    num_bkg = N - num_sig
    
    sig_selection = np.random.choice(sig_set, num_sig)
    bkg_selection = np.random.choice(bkg_set, num_bkg)
    
    mixed_sample = np.concatenate((sig_selection, bkg_selection))
    
    return mixed_sample
        
def generate_train_test_val(M1, M2, test_size = .9, val_size = 0.1):
    """
    INPUTS
    M1: np.array of data to be given the label 1
    M2: np.array of data to be given the label 0
    
    OUPUTS
    training, validation, and testing datasets + labels
    
    """

    # split data into train-test
    ((sig_train, sig_test),
     (bkg_train, bkg_test),) = [train_test_split(arr, test_size=test_size) for arr in [
                                                M1,
                                                M2,]]

    # split train data into train-val
    ((sig_train, sig_val),
     (bkg_train, bkg_val),) = [train_test_split(arr, test_size=val_size) for arr in [
                                                sig_train,
                                                bkg_train,]]

    # prepare the datasets + labels
    data_train = np.concatenate([sig_train, bkg_train])
    labels_train = np.concatenate([np.ones(sig_train.shape[0]),np.zeros(bkg_train.shape[0])])
    data_train, labels_train = shuffle(data_train, labels_train)
    data_train = np.reshape(data_train, (-1,1))
    labels_train = np.reshape(labels_train, (-1,1))

    data_val = np.concatenate([sig_val, bkg_val])
    labels_val = np.concatenate([np.ones(sig_val.shape[0]),np.zeros(bkg_val.shape[0])])
    data_val, labels_val = shuffle(data_val, labels_val)
    data_val = np.reshape(data_val, (-1,1))
    labels_val = np.reshape(labels_val, (-1,1))

    data_test = np.concatenate([sig_test, bkg_test])
    labels_test = np.concatenate([np.ones(sig_test.shape[0]),np.zeros(bkg_test.shape[0])])
    data_test, labels_test = shuffle(data_test, labels_test)
    data_test = np.reshape(data_test, (-1,1))
    labels_test = np.reshape(labels_test, (-1,1))
    
    return data_train, labels_train, data_val, labels_val, data_test, labels_test

    

## Fully supervised classifier and CWoLa classifier

### Data preparation: Fully supervised

In [ ]:
# Generate the data


N_train = 100.0 # number of training samples
N_test = 100000.0 # number of testing samples

test_frac = N_test/(N_test+N_train)
N_tot = N_train+N_test

num_sig = int(N_tot/2)
num_bkg = int(N_tot/2)


ful_sup_sig = np.random.normal(sig_mean, sig_std, num_sig)
ful_sup_bkg = np.random.normal(bkg_mean, bkg_std, num_bkg)

bins = np.linspace(-10, 30, 40)
plt.figure()
plt.hist(ful_sup_sig, bins, label = "Sig", alpha = 0.4)
plt.hist(ful_sup_bkg, bins, label = "Bkg", alpha = 0.4)
plt.legend()
plt.show()

# Make the training datasets

ful_sup_data_train, ful_sup_labels_train, ful_sup_data_val, ful_sup_labels_val, ful_sup_data_test, ful_sup_labels_test = generate_train_test_val(ful_sup_sig, ful_sup_bkg,
                                                                                                                                                test_frac, .1)  


print("Fully supervised training datasets:")
print("Training data shape:", ful_sup_data_train.shape, "Labels shape:", ful_sup_labels_train.shape)
print("Validation data shape:", ful_sup_data_val.shape, "Labels shape:", ful_sup_labels_val.shape)
print("Testing data shape:", ful_sup_data_test.shape, "Labels shape:", ful_sup_labels_test.shape)

plt.figure()
plt.hist(ful_sup_data_train, bins = 50)
plt.xlabel("Training dataset")
plt.ylabel("Counts")
plt.show()


### Data preparation: CWoLa

In [ ]:
# Generate the data

num_M1 = num_sig
num_M2 = num_bkg


sig_set = np.random.normal(sig_mean, sig_std, num_sig)
bkg_set = np.random.normal(bkg_mean, bkg_std, num_bkg)

f1 = .8
f2 = 1.0 - f1

mixed_samp_1 = generate_mixed_sample(sig_set, bkg_set, num_M1, f1)
mixed_samp_2 = generate_mixed_sample(sig_set, bkg_set, num_M2, f2)

bins = np.linspace(-10, 30, 40)
plt.figure()
plt.hist(mixed_samp_1, bins, label = "M1", alpha = 0.4)
plt.hist(mixed_samp_2, bins, label = "M2", alpha = 0.4)
plt.legend()
plt.show()

# Make the training datasets

CWoLa_data_train, CWoLa_labels_train, CWoLa_data_val, CWoLa_labels_val, CWoLa_data_test, CWoLa_labels_test = generate_train_test_val(mixed_samp_1, mixed_samp_2,
                                                                                                                                    test_frac, .1)  


print("CWoLa training datasets:")
print("Training data shape:", CWoLa_data_train.shape, "Labels shape:", CWoLa_labels_train.shape)
print("Validation data shape:", CWoLa_data_val.shape, "Labels shape:", CWoLa_labels_val.shape)
print("Testing data shape:", CWoLa_data_test.shape, "Labels shape:", CWoLa_labels_test.shape)

plt.figure()
plt.hist(CWoLa_data_train, bins = 50)
plt.xlabel("Training dataset")
plt.ylabel("Counts")
plt.show()

### Running the binary classifier: Fully supervised

In [ ]:
input_shape = 1
num_epochs = 1000
batch_size = 256
update_epochs = 10
lr = 0.0001

performance_stats = create_and_run_nn(device, input_shape, num_epochs, batch_size, update_epochs, lr, 
                      ful_sup_data_train, ful_sup_labels_train, 
                      ful_sup_data_val, ful_sup_labels_val,
                      ful_sup_data_test, ful_sup_labels_test, 
                      verbose = True, early_stop = True, LRschedule = False)


# Plot the output losses   
plt.figure()
plt.plot(performance_stats["epochs"],performance_stats["losses"], label = "loss")
plt.plot(performance_stats["val_epochs"],performance_stats["val_losses"], label = "val loss")
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.yscale("log")
plt.legend()
plt.title("Fully Supervised")
plt.show()

plt.figure()
plt.plot(performance_stats["tpr"], 1.0/performance_stats["fpr"])
plt.yscale("log")
plt.xlabel("True Positive Rate")
plt.ylabel("1/(False Positive Rate)")
plt.title("Fully Supervised")
plt.show()

print("Accuracy of the network: %d %%" % (100.00 *performance_stats["acc"]))
print("ROC AUC:", performance_stats["auc"])

full_sup_AUC = performance_stats["auc"]


### Running the binary classifier: CWoLa

In [ ]:
input_shape = 1
num_epochs = 1000
batch_size = 256
update_epochs = 10
lr = 0.0001

performance_stats = create_and_run_nn(device, input_shape, num_epochs, batch_size, update_epochs, lr, 
                      CWoLa_data_train, CWoLa_labels_train, 
                      CWoLa_data_val, CWoLa_labels_val,
                      CWoLa_data_test, CWoLa_labels_test, 
                      verbose = True, early_stop = True, LRschedule = False)


# Plot the output losses   
plt.figure()
plt.plot(performance_stats["epochs"],performance_stats["losses"], label = "loss")
plt.plot(performance_stats["val_epochs"],performance_stats["val_losses"], label = "val loss")
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.yscale("log")
plt.legend()
plt.title("CWoLa")
plt.show()

plt.figure()
plt.plot(performance_stats["tpr"], 1.0/performance_stats["fpr"])
plt.yscale("log")
plt.xlabel("True Positive Rate")
plt.ylabel("1/(False Positive Rate)")
plt.title("CWoLa")
plt.show()

print("Accuracy of the network: %d %%" % (100.00 *performance_stats["acc"]))
print("ROC AUC:", performance_stats["auc"])


### Do the CWoLa training for a range of mixed sample fractions

In [ ]:
# Always pull from the same signal / background samples (so we don't have to regenerate)


sig_set = np.random.normal(sig_mean, sig_std, num_sig)
bkg_set = np.random.normal(bkg_mean, bkg_std, num_bkg)
num_M1 = num_sig
num_M2 = num_bkg




# NN hyperparams
input_shape = 1
num_epochs = 100
batch_size = 400
update_epochs = 5
lr = 0.0001

f1_vals = []
ROC_AUC_vals = []

for f1 in np.linspace(0,1,31):
    
    f1_vals.append(f1)
    f2 = 1.0 - f1
    
    # Generate the mixed samples
    mixed_samp_1 = generate_mixed_sample(sig_set, bkg_set, num_M1, f1)
    mixed_samp_2 = generate_mixed_sample(sig_set, bkg_set, num_M2, f2)
    
    # Plot the mixed samples
    bins = np.linspace(-10, 30, 40)
    plt.figure()
    plt.hist(mixed_samp_1, bins, label = "M1", alpha = 0.4)
    plt.hist(mixed_samp_2, bins, label = "M2", alpha = 0.4)
    plt.title("f1 = "+str(f1))
    plt.legend()
    plt.show()

    # Make the training datasets
    CWoLa_data_train, CWoLa_labels_train, CWoLa_data_val, CWoLa_labels_val, CWoLa_data_test, CWoLa_labels_test = generate_train_test_val(mixed_samp_1, mixed_samp_2,
                                                                                                                                        test_frac, .1)  

    # Run the binary classifier
    
    print("Starting CWoLa training run with f1 =", f1)

    performance_stats = create_and_run_nn(device, input_shape, num_epochs, batch_size, update_epochs, lr, 
                          CWoLa_data_train, CWoLa_labels_train, 
                          CWoLa_data_val, CWoLa_labels_val,
                          CWoLa_data_test, CWoLa_labels_test, 
                          verbose = False, early_stop = True, LRschedule = False)


    # Plot the output losses   
    plt.figure()
    plt.plot(performance_stats["epochs"],performance_stats["losses"], label = "loss")
    plt.plot(performance_stats["val_epochs"],performance_stats["val_losses"], label = "val loss")
    plt.xlabel("Epochs")
    plt.ylabel("Losses")
    plt.yscale("log")
    plt.legend()
    plt.title("CWoLa")
    plt.show()

    plt.figure()
    plt.plot(performance_stats["tpr"], 1.0/performance_stats["fpr"])
    plt.yscale("log")
    plt.xlabel("True Positive Rate")
    plt.ylabel("1/(False Positive Rate)")
    plt.title("CWoLa")
    plt.show()


    
    ROC_AUC_vals.append(performance_stats["auc"])

    
    
    
    
    
    
    

In [ ]:
plt.figure()
plt.scatter(f1_vals,ROC_AUC_vals, label = "CWoLa", color = "b")
plt.scatter(f1_vals, np.full(len(f1_vals), full_sup_AUC), label = "Full. Sup.", color = "k")
plt.legend()

plt.xlabel("f1")
plt.ylabel("AUC")
plt.title("N = "+str(N_train))
plt.show()